In [ ]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
def show_image(*imgs, title=''):
    if len(imgs) > 1:
        img = np.concatenate(tuple([x for x in imgs]), axis=1)
    else:
        img = imgs[0]
    fig = plt.figure(figsize=(10,5))
    plt.imshow(img)
    fig.suptitle(title)
    
def show_prediction(prediction):
    resized = np.resize(prediction, (prediction.shape[0], prediction.shape[1]))
    show_image(resized)

In [ ]:
from models import get_detector_model, get_classification_model

In [ ]:
test_image = scipy.misc.imread('./models/data/sim_training_data/sim_data_capture/left0055.jpg')
test_image = scipy.misc.imresize(test_image, (480, 640))
show_image(test_image, title='Example Image')

In [4]:
detector_model = get_detector_model(test_image.shape[0], test_image.shape[1], 
                                    weights_file="./models/traffic_light_detector_v1.hdf5")
def get_prediction(image):
    return detector_model.predict(np.array([image]))[0]
prediction = get_prediction(test_image)
show_prediction(prediction)

(?, 480, 640, 3)
(?, 480, 640, 32)
(?, 240, 320, 32)
(?, 120, 160, 64)
(?, 60, 80, 128)
(?, 30, 40, 256)
(?, 480, 640, 1)
model 2
model 4


ResourceExhaustedError: OOM when allocating tensor with shape[1,32,480,640] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: conv2d_1/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_input_1_0_0/_241, conv2d_1/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: conv2d_19/Sigmoid/_243 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_232_conv2d_19/Sigmoid", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tenso

In [ ]:
from scipy.ndimage.measurements import label

def apply_threshold(prediction, threshold):
    prediction[prediction <= threshold] = 0
    return prediction
def generate_bounding_box(labels, label):
    nonzero = (labels == label).nonzero()
    # Identify x and y values of those pixels
    #print(sum(nonzero))
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    #if len(nonzerox) == 0 or len(nonzeroy) == 0:
    #    return None
    
    # Define a bounding box based on min/max x and y
    total_area = (np.max(nonzerox) - np.min(nonzerox)) * (np.max(nonzeroy) - np.min(nonzeroy))
    if total_area < 10:
        return None
    return ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))

def get_image_in_bounding_box(image, bbox):
    a1 = bbox[0][1]
    a2 = bbox[1][1]
    a3 = bbox[0][0]
    a4 = bbox[1][0]
#     print(f"image[{a1}:{a2},{a3}:{a4}]")
    return image[bbox[0][1]:bbox[1][1], bbox[0][0]:bbox[1][0]]
#     return image[a1:a2, a3:a4]
#     return image[bbox[0][0]:bbox[1][0],bbox[1][0]:bbox[1][1]]

prediction = apply_threshold(prediction, 0.05)

labels, number_of_lights = label(prediction)

print(f"Number of lights found: {number_of_lights}")

found_traffic_lights = []

for light_number in range(1, number_of_lights+1):
    print(light_number)
    bounding_box = generate_bounding_box(labels, light_number)
    if bounding_box is None:
        continue
    image = get_image_in_bounding_box(test_image, bounding_box)
    found_traffic_lights.append(image)
    print(bounding_box)
    show_image(image)

In [ ]:
labels = {
    'Green': 0,
    'Yellow': 1,
    'Red': 2,
    'unknown': 3
}
reverse_labels = {y: x for x,y in labels.items()}

classification_model = get_classification_model()

def predict_traffic_light(image):
    image = scipy.misc.imresize(image, (64,32))
    prediction = classification_model.predict(np.array([image]))[0]    
    correct_label = np.argmax(prediction)
    return reverse_labels[correct_label]

for image in found_traffic_lights:
    print(predict_traffic_light(image))